In [1]:
%load_ext autoreload
%autoreload 2

import pprint
import sys
import textwrap

In [2]:
import config
from dataclasses import dataclass

from llama_index.core import Document, Settings, VectorStoreIndex, ChatPromptTemplate
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.anthropic import Anthropic
from npc.prompts import NpcPrompt
from npc.prompts.prompt_common import Prompt

api_keys = config.Config("../../../api_keys.cfg")

Settings.llm = None

/home/taylor/projects/npcs/npc/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LLM is explicitly disabled. Using MockLLM.


In [3]:
anthropic_api_key = api_keys["ANTHROPIC_API_KEY"]
small_llm = Anthropic(model="claude-3-haiku-20240307", api_key=anthropic_api_key, max_tokens=4096)
large_llm = Anthropic(model="claude-3-5-sonnet-20240620", api_key=anthropic_api_key, max_tokens=4096)

In [4]:
response = small_llm.complete("What is 23 * 443.2? Concisely show your work.")
response_text = response.text
print(response_text)

23 * 443.2 = 10,173.6


# NPC Agent POC

In [5]:
from npc.interfaces import SimulatorInterface, SimulatorResponse, SimulatorRequest
from npc.llm_response_generator import LLMResponseGenerator

In [7]:
# test_working_memory = (
#     "I am walking through a forest. The trees are tall and the air is fresh. I feel a sense of peace and tranquility.",
# )
# test_observations = (
#     "I notice a squirrel running up a tree. The sunlight filters through the leaves, creating dappled patterns on the ground.",
#     "I hear the chirping of birds and the rustling of leaves in the gentle breeze.",
#     "I feel the coolness of the air on my skin and the softness of the ground beneath my feet.",
#     "I smell the earthy scent of moss and the sweet fragrance of wildflowers.",
# )

# # <query_1>peaceful forest walk</query_1>
# # <query_2>tranquil woodland experience</query_2>
# # <query_3>nature's sights and sounds</query_3>
# # <query_4>sensations of outdoor environment</query_4>
# # <query_5>childhood memories of forests</query_5>
# # <query_6>relaxing outdoor activities</query_6>
# # <query_7>wildlife in forest settings</query_7>
# # <query_8>seasonal changes in forests</query_8>
# # <query_9>favorite outdoor retreats</query_9>
# # <query_10>environmental awareness and appreciation</query_10>
# test_retrieved_memories = (
#     "I remember a peaceful forest walk I took last summer. The trees were tall and the air was fresh, creating a sense of tranquility.",
#     "I recall a childhood memory of exploring a woodland area near my home. The sights and sounds of nature left a lasting impression on me.",
#     "I have a vivid memory of a camping trip where I experienced the beauty of nature up close. The wildlife and seasonal changes were fascinating.",
#     "I think back to a favorite outdoor retreat I visited with friends. The environmental awareness and appreciation we shared was memorable.",
# )

In [8]:
# TODO: ask LLM to output in a structured reasoning template. Try to simplify the prompts (remove some of the less important items with claude's help)
#       and incorporate others into reasoning steps

# query_response_generator = LLMResponseGenerator(NpcPrompt.MEMORY_QUERY_FORMULATION.value, small_llm)
# query_response = query_response_generator.generate_response(
#     working_memory=test_working_memory,
#     observations=test_observations,
# )
# pprint.pprint(query_response, width=120)

In [9]:
# memory_report_synthesis_generator = LLMResponseGenerator(NpcPrompt.MEMORY_REPORT_SYNTHESIS.value, small_llm)
# memory_report_synthesis_response = memory_report_synthesis_generator.generate_response(
#     working_memory=test_working_memory,
#     retrieved_memories=test_retrieved_memories,
# )
# pprint.pprint(memory_report_synthesis_response, width=120)

In [10]:
# working_memory_generator = LLMResponseGenerator(NpcPrompt.WORKING_MEMORY_UPDATE.value, small_llm)
# working_memory_response = working_memory_generator.generate_response(
#     working_memory=test_working_memory,
#     memory_report=memory_report_synthesis_response,
# )
# pprint.pprint(working_memory_response, width=120)

In [11]:
# action_decision_generator = LLMResponseGenerator(NpcPrompt.ACTION_DECISION.value, small_llm)
# action_decision_response = action_decision_generator.generate_response(
#     working_memory=working_memory_response,
#     actions=[
#         "1. Explore the forest to discover new sights and sounds.",
#         "2. Sit down and meditate to deepen the sense of peace and tranquility.",
#         "3. Take out a notebook and start sketching the trees and wildlife around you.",
#     ]
# )
# pprint.pprint(action_decision_response, width=120)

In [12]:
class MemoryDatabase:
    def __init__(self, initial_memories: list[str]):
        self.index = VectorStoreIndex.from_documents(
            [Document(text=memory) for memory in initial_memories],
            embed_model=HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5"),
        )

    def add_memories(self, memories: list[str]):
        for memory in memories:
            self.index.insert(Document(text=memory))
    
    def retrieve(self, query: str, top_k: int = 5):
        retriever = VectorIndexRetriever(index=self.index, similarity_top_k=top_k)
        return retriever.retrieve(query)


@dataclass
class LLMConfig:
    small_llm: Anthropic
    large_llm: Anthropic

from npc.interfaces.text_adventure_interface import TextAdventureInterface
from typing import Generic, TypeVar

SimulatorRequestType = TypeVar('RequestType', bound=SimulatorRequest)
SimulatorResponseType = TypeVar('ResponseType', bound=SimulatorResponse)
SimulatorType = TypeVar('SimulatorType')

class Agent(Generic[SimulatorRequestType, SimulatorResponseType, SimulatorType]):
    def __init__(
            self, 
            simulator_interface: SimulatorInterface[SimulatorRequestType, SimulatorResponseType, SimulatorType],
            llm_config: LLMConfig,
            initial_working_memory: str = "",
            initial_long_term_memories: list[str] = [], # useful to add agent background information and semantic knowledge
            personality_traits: list[str] = [], # list of short, simple traits like "curious" or "optimistic"
        ):
        self.simulator_interface = simulator_interface
        self.prev_simulator_response = None

        self.working_memory = initial_working_memory
        self.long_term_memory = MemoryDatabase(initial_long_term_memories)
        self.personality_traits = personality_traits

        self.query_generator = LLMResponseGenerator(NpcPrompt.MEMORY_QUERIES.value, llm_config.small_llm)
        self.memory_report_generator = LLMResponseGenerator(NpcPrompt.MEMORY_REPORT.value, llm_config.small_llm)
        self.working_memory_generator = LLMResponseGenerator(NpcPrompt.WORKING_MEMORY.value, llm_config.small_llm)
        self.long_term_memory_generator = LLMResponseGenerator(NpcPrompt.LONG_TERM_MEMORY.value, llm_config.small_llm)
        self.action_decision_generator = LLMResponseGenerator(NpcPrompt.ACTION_DECISION.value, llm_config.small_llm)

    def update_state(self, prev_simulator_response: SimulatorResponseType) -> SimulatorRequestType:
        self.prev_simulator_response = prev_simulator_response
        self.update_working_memory()
        self.update_long_term_memory()
        # TODO: save to long term memory asynchronously to avoid blocking the main loop
        # - Unlike the Generative Agents paper, use magnitude estimation to get more accurate and actionable importance scores (will need to retrieve examples to set a baseline)

    def update_working_memory(self) -> None:
        # Formulate queries and retrieve from long-term memory
        query_response = self.query_generator.generate_response(
            working_memory=self.working_memory,
            observation=self.prev_simulator_response.observation,
        )
        retrieved_memories = []
        for query in query_response.get("queries", []):
            query_memories = self.long_term_memory.retrieve(query)
            # TODO: have some sort of LLM filtering / reranking process here
            if query_memories:
                retrieved_memories.append(query_memories[0])

        # Draft memory report based on working memory and retrieved memories
        memory_report_response = self.memory_report_generator.generate_response(
            working_memory=self.working_memory,
            observation=self.prev_simulator_response.observation,
            retrieved_memories=retrieved_memories,
        )
        print("\n".join([
            "Memory report:",
            textwrap.fill(memory_report_response["memory_report"], width=120),
            "",
        ]))

        # Update working memory based on memory report
        working_memory_response = self.working_memory_generator.generate_response(
            working_memory=self.working_memory,
            memory_report=memory_report_response["memory_report"],
        )
        if working_memory_response["updated_working_memory"]:
            self.working_memory = working_memory_response["updated_working_memory"]
        
        # TODO: remove debug print statements. Replace with loguru logging
        print("\n".join([
            "Updated working memory:",
            textwrap.fill(self.working_memory, width=120),
            "",
        ]))

    def update_long_term_memory(self) -> None:
        memory_update_response = self.long_term_memory_generator.generate_response(
            working_memory=self.working_memory,
            observation=self.prev_simulator_response.observation,
        )

        for memory in memory_update_response["memories"]:
            self.long_term_memory.add_memories([memory])


    def choose_action(self) -> SimulatorRequestType:
        next_action = self.action_decision_generator.generate_response(
            working_memory=self.working_memory,
            available_actions=self.prev_simulator_response.available_actions_llm_str(),
            action_request_documentation=self.simulator_interface.request_class().documentation_llm_str(),
        )
        print("\n".join([
            "Action decision:",
            textwrap.fill(next_action["response"], width=120),
            "",
        ]))
        # TODO: recover when invalid json causes a ValidationError
        return self.simulator_interface.request_class.parse_json(next_action["action_decision"])


# Sandbox

In [34]:
memories = [
    "I am a human.",
    "I had an apple for breakfast.",
    "My name is John.",
    "My favorite color is blue.",
    "I have a pet cat.",
    "I am 25 years old.",
    "I am a software engineer.",
]

db = MemoryDatabase(memories)

In [ ]:
query = "What did I have for breakfast?"
results = db.retrieve(query, top_k=3)
for result in results:
    print(result)

In [ ]:
db.add_memories(["I had a salad for lunch."])
results = db.retrieve("What did I have for lunch?", top_k=3)
for result in results:
    print(result)

In [ ]:
# Use LLM to generate a consistent set of memories, etc to bootstrap an agent. Cache to a text file to save time.
def bootstrap_test_agent():
    pass

# Test the agent on a set of observations. Action space can be a set of possible responses at the level of detail of a choose-your-own-adventure game.
def test_agent():
    pass

# If this turns out to be a decent evaluation methodology, could optimize the agent architecture using a genetic algorithm or other optimization technique

# Text Adventure LLM Interface

In [14]:
from npc.interfaces.text_adventure_interface import TextAdventureInterface, TextAdventureRequest, TextAdventureResponse
from npc.simulators.text_adventure import TextAdventureSimulator

story_request = "The story should involve unicorns and rainbows, but have a noir detective theme."
story_request = None

simulator = TextAdventureSimulator(small_llm, story_request)

In [15]:
# TODO: generalize this so that it does not have any text adventure specific code. The only mixing of simulator logic and agent logic should be in the interface

def run_text_adventure_simulation(simulator: TextAdventureSimulator, config: LLMConfig, max_steps: int = 3):
    simulator_interface = TextAdventureInterface(simulator)
    agent = Agent(simulator_interface=simulator_interface, llm_config=config)

    # TODO: don't touch simulator except through the interface
    simulator_response = TextAdventureResponse(
        success=True,
        message="Initial state",
        observation=simulator.state.observation,
        available_actions=simulator.state.available_actions,
    )

    for _ in range(max_steps):
        print(200 * "=")
        print("\n".join([
            "Observation:",
            textwrap.fill(simulator_response.observation, width=120),
            "",
        ]))
        agent.update_state(simulator_response)
        simulator_request = agent.choose_action()
        simulator_response = simulator_interface.execute(simulator_request)
        
        if not simulator_response.success:
            print(f"Error: {simulator_response.message}")
            break
        
        if simulator.is_story_ended():
            print("Story has reached its conclusion.")
            break

    print("Adventure completed!")

run_text_adventure_simulation(simulator, LLMConfig(small_llm, small_llm), max_steps=2)

Observation:
The neon-soaked streets of the city pulsed with a frenetic energy as I made my way through the crowded alleyways, my
footsteps echoing against the towering walls of the corporate skyscrapers that loomed overhead. The information I had
uncovered was weighing heavily on my mind, a tangled web of corruption and deceit that stretched from the shadowy
underworld all the way up to the highest echelons of the megacorporation.  As I ducked into a dimly lit bar, the
familiar scent of stale cigarette smoke and cheap liquor assaulted my senses. This was where I had arranged to meet my
contact, a former corporate drone turned underground informant. I scanned the room, my eyes quickly settling on a figure
hunched over a glass of amber liquid in the far corner.  Approaching cautiously, I slid into the booth across from the
informant, my gaze locked onto their face, searching for any sign of unease or betrayal. "You said you had something for
me," I murmured, my voice low and guarded.  T

In [ ]:
# TODO: move most of the code out of __init__.py and into separate files

# Text Adventure Human Interface

In [7]:
from rich.console import Console
from rich.markdown import Markdown
from rich.panel import Panel
from rich.prompt import IntPrompt

from npc.simulators.text_adventure import TextAdventureSimulator

console = Console()

story_request = "Give me a story about life as a house cat named banshee, who is extremely food motivated. She lives in a two bedroom apartment with her owners, Jenny and Taylor, who are engaged. They are starting to plan their wedding, and banshee wants to take advantage of this to get treats and food. End the story if she gets treats."
# story_request = None
console.print(Panel(Markdown(story_request), title="Story Request"))
# simulator = TextAdventureSimulator(small_llm, story_request)
simulator = TextAdventureSimulator(large_llm, story_request)
console.print(Panel(Markdown(simulator.story_guide), title="Story Guide"))

╭───────────────────────────────────────────────── Story Request ─────────────────────────────────────────────────╮
│ Give me a story about life as a house cat named banshee, who is extremely food motivated. She lives in a two    │
│ bedroom apartment with her owners, Jenny and Taylor, who are engaged. They are starting to plan their wedding,  │
│ and banshee wants to take advantage of this to get treats and food. End the story if she gets treats.           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Story so far: 
Story section: I stretch lazily on my favorite sunspot, my whiskers twitching as I take in the new scents wafting through the apartment. Something's different today. The air is thick with excitement and... is that a hint of stress? My humans, Jenny and Taylor, are bustling about more than usual, their voices pitched higher than normal.

I slink off my perch and pad towards the kitchen, my true domain. As I round the corner, I'm greeted by a sight that makes my tail stand straight up. The big cold food box (what humans call a "refrigerator") is plastered with colorful papers. Curious, I leap onto the counter for a closer look.

The papers are covered in squiggles that mean nothing to me, but the pictures catch my eye. Flowers, cakes, and humans in fancy fur-less coverings. What could it all mean? More importantly, will it lead to more treats for me?

Just then, Jenny rushes into the kitchen, her face flushed and her hair a mess. "Taylor! Have you seen the caterer's number

╭────────────────────────────────────────────────── Story Guide ──────────────────────────────────────────────────╮
│ Title: "The Feline Wedding Planner"                                                                             │
│                                                                                                                 │
│  1 Genre and Setting:                                                                                           │
│                                                                                                                 │
│  • Genre: Humorous slice-of-life with elements of strategy                                                      │
│  • Setting: A cozy two-bedroom apartment in a modern city                                                       │
│                                                                                                                 │
│  2 Worldbuilding:                                                                                               │
│                                                                                                                 │
│  • The apartment is on the third floor of a building with large windows, perfect for Banshee to observe the     │
│    outside world                                                                                                │
│  • Jenny and Taylor's apartment is filled with wedding magazines, sample invitations, and fabric swatches       │
│  • The kitchen is Banshee's favorite room, with a treat jar placed tantalizingly on the counter                 │
│  • Banshee has a cat tree near the window and a comfortable bed in the living room                              │
│                                                                                                                 │
│  3 Style and Tone:                                                                                              │
│                                                                                                                 │
│  • First-person narrative from Banshee's perspective                                                            │
│  • Humorous and slightly sarcastic tone, reflecting Banshee's food-obsessed mindset                             │
│  • Short, punchy sentences to mimic a cat's attention span                                                      │
│  • Use of cat-related metaphors and similes                                                                     │
│                                                                                                                 │
│  4 Central Conflict/Goals:                                                                                      │
│                                                                                                                 │
│  • Primary Goal: Banshee wants to maximize her treat intake during the wedding planning process                 │
│  • Secondary Conflict: Banshee must navigate the changing dynamics in the household as Jenny and Taylor become  │
│    more stressed and busy                                                                                       │
│                                                                                                                 │
│  5 Main Character:                                                                                              │
│                                                                                                                 │
│  • Banshee: A food-motivated house cat with a mischievous streak                                                │
│  • Initial Situation: Banshee notices increased activity and stress in the household, realizing it's related to │
│    wedding planning                                                                                             │
│                                                       

In [10]:
def print_wrapped_text(text: str):
    console.print(Markdown(text))

while not simulator.is_story_ended():
    state = simulator.state

    # Display the current game state
    console.rule("Game State", style="cyan")
    print_wrapped_text(state.observation)

    # Display available actions
    action_text = "\n\n".join([f"**Option {i}**  \n{action}" for i, action in state.available_actions.items()])
    actions_panel = Panel(Markdown(action_text), title="[bold cyan]Available Actions[/bold cyan]", border_style="bright_blue")
    console.print(actions_panel)

    # Take an action
    action_index = IntPrompt.ask("Choose an action", choices=[str(i) for i in state.available_actions.keys()])
    console.print(f"\nYou chose: [bold green]Option {action_index}[/bold green]\n")
    new_game_state = simulator.take_action(action_index)

─────────────────────────────────────────────────── Game State ────────────────────────────────────────────────────

I launch into a series of loud, demanding meows, expressing my hunger and desire for treats. My vocal performance  
echoes through the kitchen, causing Jenny to wince and cover her ears.                                             

"Banshee, please! Not now!" Jenny pleads, her stress levels visibly rising.                                        

Before I can escalate my feline aria, Taylor swoops in, scooping me up in his arms. "I've got this, honey," he says
to Jenny, who shoots him a grateful look.                                                                          

Taylor carries me out of the kitchen and into the living room, depositing me on my cat tree near the window. "There
you go, princess. How about a change of scenery?"                                                                  

From my new vantage point, I observe the wedding preparations unfolding. The coffee table is covered in magazines, 
and Taylor settles down on the couch, rifling through them. I watch, intrigued, as he pulls out a notepad and      
starts scribbling.                                                                                                 

The room smells of paper, ink, and a faint whiff of Taylor's cologne. Outside, birds chirp tantalizingly, but I'm  
more interested in the potential for treats in this new situation. Taylor looks less frazzled than Jenny, and I    
wonder if I can use this to my advantage.                                                                          

I notice a small plate of cheese cubes on the side table – clearly, the humans are snacking while they work. My    
whiskers twitch with interest. Perhaps there's more than one way to get a tasty morsel around here.                

As I contemplate my next move, I realize that this "wedding" thing might just be the key to unlocking a treasure   
trove of treats. But how to proceed? That is the question that now occupies my feline mind.

╭─────────────────────────────────────────────── Available Actions ───────────────────────────────────────────────╮
│ Option 1                                                                                                        │
│ Stealth Mission. Attempt to sneak down from the cat tree and swipe a cheese cube without Taylor noticing.       │
│                                                                                                                 │
│ Option 2                                                                                                        │
│ Charm Offensive. Jump down and curl up next to Taylor, purring loudly and being extra affectionate to win his   │
│ favor.                                                                                                          │
│                                                                                                                 │
│ Option 3                                                                                                        │
│ Playful Distraction. Bat at the pages of the magazine Taylor is reading, trying to engage him in a game and     │
│ divert his attention from work.                                                                                 │
│                                                                                                                 │
│ Option 4                                                                                                        │
│ Window Watcher. Stay on the cat tree and intently watch the birds outside, occasionally making chittering       │
│ noises to draw Taylor's attention and sympathy.                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Choose an action [1/2/3/4]:

You chose: Option 4

...

I settle into my perch on the cat tree, my eyes fixed on the fluttering feathered morsels outside. My tail twitches
with excitement as I watch a particularly plump sparrow hop along the windowsill. I can't help but let out a series
of soft, rapid chitters – a sound that always seems to amuse my humans.                                            

As expected, my performance catches Taylor's attention. He looks up from his pile of shiny paper squares           
(invitations, I think they're called) and chuckles. "Aw, is someone feeling a little predatory today?"             

I respond with another round of chittering, my eyes never leaving the tantalizing bird buffet just beyond my reach.
Taylor's face softens, and he reaches for the plate on the side table. My nose twitches as I catch a whiff of      
something deliciously savory.                                                                                      

"Here you go, mighty huntress," Taylor says, holding out a small cube of cheese. "A reward for keeping our home    
safe from those terrifying sparrows."                                                                              

I gracefully leap down from my perch and delicately accept the offering. The cheese is creamy and rich, melting on 
my tongue. It's not my usual treat, but I'm not one to turn down any form of sustenance. As I savor my snack, I    
notice Taylor's shoulders relax a bit. He reaches out to scratch behind my ears, and I lean into his touch, purring
contentedly.                                                                                                       

"You know, Banshee," Taylor muses, "sometimes I wish planning this wedding was as simple as watching birds. No     
caterers to wrangle, no seating charts to agonize over..."                                                         

I blink up at him, my whiskers twitching. Humans and their strange rituals. Why can't they just share a mouse and  
be done with it?                                                                                                   

As Taylor returns to his task, I notice the cheese plate is still within reach. My treat was delicious, but one    
cube hardly constitutes a meal. Perhaps there's an opportunity here to score more snacks while simultaneously      
providing some much-needed stress relief to my favorite treat-dispensers.                                          

The question is, how should I proceed? The cheesy treasure is tempting, but so is the possibility of more attention
– and potential treats – from Taylor. And let's not forget Jenny in the kitchen. She might be more inclined to     
share if I offer my "assistance" with her tasks.                                                                   

Decisions, decisions. A cat's work is never done when there are treats to be earned and humans to be managed.

╭─────────────────────────────────────────────── Available Actions ───────────────────────────────────────────────╮
│ Option 1                                                                                                        │
│ Cheese Heist. Attempt to sneak another piece of cheese from the plate while Taylor is distracted with           │
│ invitations.                                                                                                    │
│                                                                                                                 │
│ Option 2                                                                                                        │
│ Feline Therapy. Curl up next to Taylor, offering comforting purrs and head-butts to help him relax and          │
│ potentially earn more treats.                                                                                   │
│                                                                                                                 │
│ Option 3                                                                                                        │
│ Kitchen Reconnaissance. Investigate Jenny's activities in the kitchen, looking for opportunities to "help" and  │
│ possibly score some different treats.                                                                           │
│                                                                                                                 │
│ Option 4                                                                                                        │
│ Playful Distraction. Initiate a game with one of the loose invitation envelopes, providing entertainment and a  │
│ much-needed break for Taylor.                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Choose an action [1/2/3/4]:

You chose: Option 1

KeyboardInterrupt: 

In [7]:
def print_story_nodes(simulator):
    nodes = simulator.current_node.path_from_root()
    for i, node in enumerate(nodes):
        heading = f"Node {i + 1}"
        llm_output = node.llm_response
        print("\n".join([
            f"{heading}",
            "=" * len(heading),
            llm_output["response"],
            "",
        ]))

print_story_nodes(simulator)

Node 1
<story_continuation_reasoning>
Since this is the beginning of the story, we should continue rather than end it. We need to establish the initial situation, introduce the main characters, and set up the central conflict. Ending the story now would be premature and unsatisfying for the reader. Continuing allows us to develop Banshee's character, introduce the wedding planning scenario, and present the first set of choices for the reader.
</story_continuation_reasoning>

<next_section>
Purr-fect day for a catnap. Sun puddle on the windowsill, warm and cozy. But wait—what's this? My humans, Jenny and Taylor, are making excited noises. More excited than usual treat time noises. Ears perk up, whiskers twitch. Time to investigate.

I slink into the living room, tail held high. Jenny's waving her paw around, something sparkly on it. Taylor's grinning like he just caught the red dot. Hmmm. Suspicious.

"We're engaged!" Jenny squeals. Engaged? Is that some kind of new treat? My stomach ru